In [1]:
import pandas as pd
import csv
from loader import *
from pair_finder import *
import time
from random import seed
import keyboard
from random import seed
from random import randint

In [2]:
ratings_path_s = "DATA\\ratings_100users_shuffled.csv"
ratings_path = "DATA\\ratings_100users.csv"
movies_path =  "DATA\\movies.csv"

MinScore = 4
MaxCombo = 5
MinFrequency = 0.1
ratings_stream = createRatingsStream(ratings_path, MinScore)
userBaskets, movie_list = CreateMovieBaskets(ratings_path, MinScore, 10)
movies_df = ReadMovies(movies_path, movie_list)

#combos = myApriori(userBaskets, 0.3, 5)

In [7]:

tm = TriangularMatrixOfPairsCounters(movies_df, userBaskets)

User:  1 / 100
User:  2 / 100
User:  3 / 100
User:  4 / 100
User:  5 / 100
User:  6 / 100
User:  7 / 100
User:  8 / 100
User:  9 / 100
User:  10 / 100


In [14]:
tm[get_tm_pos((3, 1), movies_df)]

1

In [9]:
combo_dict = HashCountersOfPairs(userBaskets)

User:  1 / 100
User:  2 / 100
User:  3 / 100
User:  4 / 100
User:  5 / 100
User:  6 / 100
User:  7 / 100
User:  8 / 100
User:  9 / 100
User:  10 / 100


In [13]:
combo_dict[(3,1)]

1

In [ ]:
mycombos = export_combos(myApriori(userBaskets, MinFrequency, MaxCombo))

In [ ]:
def sampledApriori(sample_size, ratings_stream, min_frequency, max_length):
    SetOfUsers = [] # User so far
    sampleOfBaskets = {} # Key: userId, Value: movie_basket
    sample_map = [] # Mapping between sample index and userId
    for i in range(len(ratings_stream)):

        if (keyboard.is_pressed('y') or keyboard.is_pressed('Y')) and i > len(ratings_stream):
            combos_dict = myApriori(list(sampleOfBaskets.values()), min_frequency, max_length)
            combos = export_combos(combos_dict)

            return combos, sampleOfBaskets
        else:
            current_assessment = ratings_stream.iloc[i]
            current_user = int(current_assessment['userId'])
            current_movie = int(current_assessment['movieId'])

            if current_user not in SetOfUsers:
                SetOfUsers.append(current_user)
                sampleOfBaskets = reservoir_sampling(len(SetOfUsers), sample_size, sampleOfBaskets, sample_map, current_user)

            if current_user in sampleOfBaskets.keys():
                sampleOfBaskets[current_user].append(current_movie)
    
    # do first apriori here
    #return sampleOfBaskets.values()
    combos_dict_1 = myApriori(list(sampleOfBaskets.values()), min_frequency, max_length)
    combos_1 = export_combos(combos_dict_1)

    SetOfUsers = []
    sampleOfBaskets = {}
    sample_map = [] 
    # Second run
    for i in range(len(ratings_stream)):
        current_assessment = ratings_stream.iloc[i]
        current_user = int(current_assessment['userId'])
        current_movie = int(current_assessment['movieId'])

        if current_user not in SetOfUsers:
            SetOfUsers.append(current_user)
            sampleOfBaskets = reservoir_sampling(len(SetOfUsers), sample_size, sampleOfBaskets, sample_map, current_user)

        if current_user in sampleOfBaskets.keys():
            sampleOfBaskets[current_user].append(current_movie)
    
    combos_dict_2 = myApriori(list(sampleOfBaskets.values()), min_frequency, max_length)
    combos_2 = export_combos(combos_dict_2)

    return [c1 for c1 in combos_1 if c1 in combos_2]

def reservoir_sampling(n_distinct_users, sample_size, sample_of_baskets, sample_map, current_user):
    seed()
    if n_distinct_users <= sample_size:
        sample_of_baskets[current_user] = []
        sample_map.append(current_user)
    else:
        tmp = randint(0, n_distinct_users)

        if tmp < sample_size:
            old_user = sample_map[tmp]
            sample_map[tmp] = current_user
            del sample_of_baskets[old_user]
            sample_of_baskets[current_user] = []
            

    return sample_of_baskets


def export_combos(combos_dict):
    combos = []
    for i in combos_dict.keys():
        for j in combos_dict[i].keys():
            combos.append(j)
    return combos

In [ ]:
combos, sd= sampledApriori(50, ratings_stream, MinFrequency, MaxCombo)

In [ ]:
sd.keys()

In [ ]:
tp = []
fp = []
fn = []
for i in mycombos:
    if i in combos:
        tp.append(i)
    else:
        fn.append(i)
for i in combos:
    if i not in mycombos:
        fp.append(i)
tp = len(tp)
fp = len(fp)
fn = len(fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
x = {}
k = (1,2)
x[k] = 4

In [ ]:
x[(1,2)]